Let's construct a parametrizable nn architecture.
Here we encode the genotype as a simple string of zeros and ones and we construct a simple neural network with one ore more hidden layers each one made of a random number of nodes.

In [4]:
from random import random, randint, seed
from statistics import mean
from copy import deepcopy

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

import numpy as np
import math
import copy


import torchvision
import torchvision.transforms as transforms
import os
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import PIL.Image as Image
from torchvision import datasets,models
import matplotlib.pyplot as plt

In [5]:
class Net(torch.nn.Module):
    def __init__(self, genotype):
        super().__init__()
        self.layerlist = []
        self.input_size = 6
        self.output_size = 1
        self.curr_input_size = copy.deepcopy(self.input_size)

        for gene in genotype: 

            # add final layer
            if gene == 0:
                self.layerlist.append(torch.nn.Linear(in_features=self.curr_input_size, out_features=self.output_size))
                self.layerlist.append(torch.nn.ReLU())

            # add another intermediate layer
            elif gene == 1:
                n_nodes = np.random.randint(1, 10)
                self.layerlist.append(torch.nn.Linear(in_features=self.curr_input_size, out_features=n_nodes))
                self.layerlist.append(torch.nn.ReLU())

                self.curr_input_size = n_nodes
                
    def forward(self, X):
        self.layerlist.append(torch.nn.Sigmoid())
        self.layers = torch.nn.Sequential(*self.layerlist)
        out = self.layers(X)
        
        return out

In [ ]:
torch.manual_seed(123456)

X = torch.rand(1,10,6)

model=Net([1,1,1,0])
# feed data into the model
y = model.forward(X)

summary(model)

Now we use the same construction to build a CNN (like LeNet) adding a customizable number of inner blocks, each one with a random number for output channels and kernel size.

So that the CNNs we obtain differ in:
* length
* type of convolutional layer apply:
    * output channels (input ch. as well)
    * kernel size

In [5]:
# prepare a preprocessing pipeline 
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
])

# We download the train and the test dataset in the given root and applying the given transforms
trainset = torchvision.datasets.MNIST(root='./data', train=True,  download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False,  download=True, transform=transform)

batch_size=4

# dataloaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,  shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,  shuffle=False, num_workers=2)

In [62]:
class ConvNet(nn.Module):
    def __init__(self, genotype):
        super().__init__()
        self.layerlist = []
        self.input_channels = 1
        self.output_channels = 6
        self.output_size = 10
        self.kernel_size = 5
        self.input_size = 28

        for gene in genotype: 
            # add final layer
            if gene == 0:
                self.layerlist.append(torch.nn.Flatten())
                size = (self.input_size)**2
                self.layerlist.append(torch.nn.Linear(size, 120))
                self.layerlist.append(torch.nn.ReLU())
                self.layerlist.append(torch.nn.Linear(120, 84))
                self.layerlist.append(torch.nn.ReLU())
                self.layerlist.append(torch.nn.Linear(84, self.output_size))

            # add another intermediate layer
            elif gene == 1:
                output_channels = np.random.randint(1, 10)
                kernel_size = np.random.randint(2, 5)
                self.layerlist.append(torch.nn.Conv2d(in_channels=self.input_channels, out_channels=output_channels, kernel_size=kernel_size))
                self.layerlist.append(torch.nn.ReLU())
                self.layerlist.append(torch.nn.MaxPool2d(2, 2))
                self.input_size = int((self.input_size - kernel_size + 1)/2)
                self.output_channels = output_channels
                self.input_channels = output_channels
                self.kernel_size = kernel_size


    def forward(self, x):
        self.layers = torch.nn.Sequential(*self.layerlist)
        out = self.layers(x)
        return out

In [23]:
trainloader.dataset.data[0].unsqueeze(0).shape 

torch.Size([1, 28, 28])

In [68]:
torch.manual_seed(123456)

model=ConvNet([1,1,0])
# feed data into the model
x = trainloader.dataset.data[0].unsqueeze(0).float()
y = model.forward(x)

summary(model, input_size=(1, 1, 28, 28))

Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       60
|    └─ReLU: 2-2                         --
|    └─MaxPool2d: 2-3                    --
|    └─Conv2d: 2-4                       175
|    └─ReLU: 2-5                         --
|    └─MaxPool2d: 2-6                    --
|    └─Flatten: 2-7                      --
|    └─Linear: 2-8                       4,440
|    └─ReLU: 2-9                         --
|    └─Linear: 2-10                      10,164
|    └─ReLU: 2-11                        --
|    └─Linear: 2-12                      850
Total params: 15,689
Trainable params: 15,689
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       60
|    └─ReLU: 2-2                         --
|    └─MaxPool2d: 2-3                    --
|    └─Conv2d: 2-4                       175
|    └─ReLU: 2-5                         --
|    └─MaxPool2d: 2-6                    --
|    └─Flatten: 2-7                      --
|    └─Linear: 2-8                       4,440
|    └─ReLU: 2-9                         --
|    └─Linear: 2-10                      10,164
|    └─ReLU: 2-11                        --
|    └─Linear: 2-12                      850
Total params: 15,689
Trainable params: 15,689
Non-trainable params: 0